In [1]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 857.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 42.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c8e9edd99cf1fabdb7823e458c41b49118b422411fb53fdcac0d5de44f2f26c1
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c

In [2]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

In [4]:
args = ClassificationDeployArguments(
    pretrained_model_name = "beomi/kcbert-base",
    downstream_model_dir = "/content/drive/MyDrive/class/language/checkpoint-doccls",
    max_seq_length = 128
)

downstream_model_checkpoint_fpath: /content/drive/MyDrive/class/language/checkpoint-doccls/epoch=0-val_loss=0.27.ckpt


In [5]:
from transformers import BertTokenizer

In [6]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [7]:
import torch

In [8]:
fined_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu')
)

In [10]:
from transformers import BertConfig

In [20]:
pretrained_bert_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fined_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel()
)

In [21]:
fined_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel()

2

In [22]:
from transformers import BertForSequenceClassification

In [23]:
model = BertForSequenceClassification(pretrained_bert_config)

In [24]:
model.load_state_dict({k.replace('model.',""):v for k,v in fined_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [ ]:
fined_tuned_model_ckpt['state_dict'].items()

In [27]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [44]:
def inference_fn(sentence) :
  inputs = tokenizer(
      [sentence],
      max_length = args.max_seq_length,
      padding = "max_length",
      truncation = True,
      return_tensors="pt"
  )

  with torch.no_grad():
    outputs = model(**inputs)
    prob = outputs.logits.softmax(dim=-1)
    # print(prob[0][1].item())
    positive_prob = round(prob[0][1].item(),4)
    negative_prob = round(prob[0][0].item(),4)
    pred ="긍정(positive)" if torch.argmax(prob) == 1 else "부정(negative)"
    # print(pred, positive_prob)
  
  return {
      'sentence':sentence,
      'prediction':pred,
      'positive_data': f"긍정 {positive_prob}",
      'negative_data': f"부정 {negative_prob}",
      'positive_width': f"{positive_prob * 100}%",
      'negative_width': f"{negative_prob * 100}%",
  }

In [61]:
inference_fn('ㅋㅋㅋ')

{'sentence': 'ㅋㅋㅋ',
 'prediction': '긍정(positive)',
 'positive_data': '긍정 0.5517',
 'negative_data': '부정 0.4483',
 'positive_width': '55.169999999999995%',
 'negative_width': '44.83%'}

In [69]:
!mkdir /root/.ngrok2 && echo "authtoken: 2KUKWK7BGUqKWYtcmI6OA1BQHR3_73E9abYcCoNbPq4kqDvJb" > /root/.ngrok2/ngrok.yml

In [68]:
# !rm -r /root/.ngrok2

In [70]:
from ratsnlp.nlpbook.classification import get_web_service_app

In [71]:
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://abf7-35-189-189-255.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:42:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:42:38] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:45:42] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:45:47] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:45:52] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:45:56] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:46:01] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:46:04] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:46:08] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:46:59] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:47:11] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:47:25] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2023 06:47:33] "POST /api